In [1]:
import osmnx as ox
import requests
import pandas as pd
import folium

In [2]:
# Get the coordinates for Dublin, Ireland
location = ox.geocode('Dublin, Ireland')

In [3]:
location

(53.3498006, -6.2602964)

In [4]:
def get_charging_stations(latitude, longitude, radius, api_key):
    url = f"https://api.openchargemap.io/v3/poi/?output=json&latitude={latitude}&longitude={longitude}&distance={radius}&distanceunit=KM&maxresults=300&&verbose=false&key={api_key}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data

    except requests.exceptions.RequestException as e:
        print("Error occurred while retrieving charging station data:", e)
        return None
    except ValueError as e:
        print("Error occurred while parsing JSON response:", e)
        return None

# Dublin Center coordinates
latitude = location[0]
longitude = location[1]

# Radius in kilometers
radius = 25

# API key from OpenChargeMap
api_key = "431e2548-cd50-4959-971b-5e99539c2651"

charging_stations = get_charging_stations(latitude, longitude, radius, api_key)

if charging_stations:
    # Convert charging_stations to DataFrame
    df_charging_stations = pd.DataFrame(charging_stations)

    # Save DataFrame to CSV
    df_charging_stations.to_csv('02_charging_stations_initial.csv', index=False)

In [5]:
len(df_charging_stations)

244

In [6]:
df_charging_stations.head()

,DataProvider,OperatorInfo,UsageType,SubmissionStatus,IsRecentlyVerified,DateLastVerified,ID,UUID,DataProviderID,DataProvidersReference,...,DateCreated,SubmissionStatusTypeID,StatusType,OperatorsReference,UsageCost,NumberOfPoints,StatusTypeID,GeneralComments,UserComments,MediaItems
0,{'WebsiteURL': 'http://chargepoints.dft.gov.uk...,"{'WebsiteURL': 'https://ebcharging.co.uk/', 'C...","{'IsPayAtLocation': True, 'IsMembershipRequire...","{'IsLive': True, 'ID': 100, 'Title': 'Imported...",True,2023-05-04T08:52:00Z,214705,EE003D02-8575-420F-8797-C9B2D7D546C2,18,153a719109d5901f816d699f83ba381e,...,2023-02-09T08:13:00Z,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'WebsiteURL': 'http://openchargemap.org', 'Da...",{'WebsiteURL': 'https://www.gocharge.ie/charge...,"{'IsPayAtLocation': False, 'IsMembershipRequir...","{'IsLive': True, 'ID': 200, 'Title': 'Submissi...",False,2020-08-08T15:24:00Z,127304,672CF633-6914-4CB9-9978-731411CF78B4,1,NaN,...,2019-06-03T08:42:00Z,200,"{'IsOperational': True, 'IsUserSelectable': Tr...",C8PJ7,€1 Connection Fee & €0.30 per kWh,2.0,50.0,NaN,NaN,NaN
2,"{'WebsiteURL': 'http://openchargemap.org', 'Da...","{'WebsiteURL': 'http://www.teslamotors.com', '...","{'IsPayAtLocation': False, 'IsMembershipRequir...","{'IsLive': True, 'ID': 200, 'Title': 'Submissi...",False,2018-10-11T13:54:00Z,109200,94158280-CA4C-4187-BF49-E860D5C73A9E,1,NaN,...,2018-10-11T13:54:00Z,200,"{'IsOperational': True, 'IsUserSelectable': Tr...",44550,NaN,2.0,50.0,"2 Tesla Connectors, up to 6kW.Available by res...",NaN,NaN
3,"{'WebsiteURL': 'http://openchargemap.org', 'Da...",{'WebsiteURL': 'http://www.esb.ie/electric-car...,"{'IsPayAtLocation': False, 'IsMembershipRequir...","{'IsLive': True, 'ID': 100, 'Title': 'Imported...",False,2021-12-29T16:32:00Z,66609,CAD14D5B-7DED-4534-A10A-650354BD9C09,1,NaN,...,2016-05-23T16:08:00Z,100,"{'IsOperational': True, 'IsUserSelectable': Tr...",C6WKW,NaN,2.0,50.0,NaN,NaN,NaN
4,"{'WebsiteURL': 'http://openchargemap.org', 'Da...",{'Comments': 'For use when the operator of the...,"{'IsPayAtLocation': False, 'IsMembershipRequir...","{'IsLive': True, 'ID': 100, 'Title': 'Imported...",False,2017-02-09T22:09:00Z,15102,B531CB42-F2C5-4C9A-99AD-94F6D02DEE89,1,NaN,...,2012-12-11T15:08:00Z,100,"{'IsOperational': True, 'IsUserSelectable': Tr...",The Westin Dublin,NaN,1.0,50.0,NaN,NaN,NaN


Based on looking at the data, we saw that the columns 'AddressInfo' and 'Connections' contain the relevant information we need for our analysis. So we'll pick them and use them for further processing

In [14]:
df_1 = df_charging_stations.loc[:, ['UUID', 'AddressInfo']]
df_2 = df_charging_stations.loc[:, ['UUID', 'Connections']]

In [15]:
len(df_1)

244

In [16]:
len(df_2)

244

In [17]:
# Get the UUID value where 'Connections' column is empty
uuids_to_exclude = df_2.loc[df_2['Connections'].apply(lambda x: len(x) == 0), 'UUID'].values

# Remove the records in df_2 without any connection info
df_2 = df_2[df_2['Connections'].apply(lambda x: len(x) != 0)]

# Remove the corresponding records from df_1 as well
df_1 = df_1[~df_1['UUID'].isin(uuids_to_exclude)]

In [19]:
len(df_1)

243

In [20]:
len(df_2)

243

After analyzing the data, we saw that certain locations have multiple charging ports within the same spot. To handle this scenario, the 'Connections' column, which contains a dictionary data type with multiple keys representing each charging port, can be split into individual records. The next section does exactly that:

In [21]:
arr1 = []

for index, row in df_1.iterrows():
    # Access values of each column for the current row
    val1 = row['UUID']
    val2 = [row['AddressInfo']]
    
    for dictionary in val2:
        dictionary['UUID'] = val1
        arr1.append(dictionary)

arr1 = pd.DataFrame(arr1)

In [22]:
arr1.head()

,ID,Title,AddressLine1,Town,Postcode,CountryID,Country,Latitude,Longitude,AccessComments,RelatedURL,Distance,DistanceUnit,UUID,StateOrProvince,ContactTelephone1,AddressLine2,ContactTelephone2,ContactEmail
0,215085,The Spire Car Park 1,"Marlborough Place, North City",Dublin,D01 W207,1,"{'ISOCode': 'GB', 'ContinentCode': 'EU', 'ID':...",53.352036,-6.258841,,https://ebcharging.co.uk/,0.266676,1,EE003D02-8575-420F-8797-C9B2D7D546C2,NaN,NaN,NaN,NaN,NaN
1,127650,Q-Park The Spire Multi-Storey Car Park,Cathal Brugha Street,Dublin 1,NaN,3,"{'ISOCode': 'IE', 'ContinentCode': 'EU', 'ID':...",53.352022,-6.258432,Located on ground floor of car park,http://www.gocharge.ie,0.276272,1,672CF633-6914-4CB9-9978-731411CF78B4,County Dublin,NaN,NaN,NaN,NaN
2,109546,Cassidys Hotel,Cavendish Rowupper O'connell Street,City Centre,D01 V3P6,3,"{'ISOCode': 'IE', 'ContinentCode': 'EU', 'ID':...",53.353102,-6.261599,NaN,NaN,0.377143,1,94158280-CA4C-4187-BF49-E860D5C73A9E,Dublin,+353 1 87 80 555,NaN,NaN,NaN
3,66955,Parnell Square West,Parnell Square West,Dublin 1,NaN,3,"{'ISOCode': 'IE', 'ContinentCode': 'EU', 'ID':...",53.353218,-6.264812,NaN,NaN,0.483969,1,CAD14D5B-7DED-4534-A10A-650354BD9C09,County Dublin,NaN,NaN,NaN,NaN
4,15448,The Westin Dublin,"The Westin Dublin, Westmoreland Street, Dublin 2",Dublin,NaN,3,"{'ISOCode': 'IE', 'ContinentCode': 'EU', 'ID':...",53.345479,-6.258945,NaN,NaN,0.488841,1,B531CB42-F2C5-4C9A-99AD-94F6D02DEE89,NaN,NaN,NaN,NaN,NaN


In [23]:
len(arr1)

243

In [24]:
arr2 = []

for index, row in df_2.iterrows():
    # Access values of each column for the current row
    val1 = row['UUID']
    val2 = row['Connections']
    val3 = val2[0]['ConnectionType']
    
    for dictionary in val2:
        dictionary['UUID'] = val1
        dictionary['Name'] = val3['Title']
        arr2.append(dictionary)
        
arr2 = pd.DataFrame(arr2)

In [25]:
arr2.head()

,ID,ConnectionTypeID,ConnectionType,Reference,StatusTypeID,StatusType,LevelID,Level,Amps,Voltage,PowerKW,CurrentTypeID,CurrentType,UUID,Name,Quantity,Comments
0,357365,25,"{'FormalName': 'IEC 62196-2 Type 2', 'IsDiscon...",1837-01-01,50,"{'IsOperational': True, 'IsUserSelectable': Tr...",2.0,"{'Comments': 'Over 2 kW, usually non-domestic ...",32.0,400.0,22.0,20.0,{'Description': 'Alternating Current - Three P...,EE003D02-8575-420F-8797-C9B2D7D546C2,Type 2 (Socket Only),NaN,NaN
1,178256,25,"{'FormalName': 'IEC 62196-2 Type 2', 'IsDiscon...",NaN,50,"{'IsOperational': True, 'IsUserSelectable': Tr...",2.0,"{'Comments': 'Over 2 kW, usually non-domestic ...",32.0,400.0,22.0,20.0,{'Description': 'Alternating Current - Three P...,672CF633-6914-4CB9-9978-731411CF78B4,Type 2 (Socket Only),2.0,NaN
2,154318,30,"{'IsDiscontinued': False, 'IsObsolete': False,...",NaN,50,"{'IsOperational': True, 'IsUserSelectable': Tr...",2.0,"{'Comments': 'Over 2 kW, usually non-domestic ...",NaN,NaN,6.0,10.0,{'Description': 'Alternating Current - Single ...,94158280-CA4C-4187-BF49-E860D5C73A9E,Tesla (Model S/X),2.0,NaN
3,92032,25,"{'FormalName': 'IEC 62196-2 Type 2', 'IsDiscon...",NaN,50,"{'IsOperational': True, 'IsUserSelectable': Tr...",2.0,"{'Comments': 'Over 2 kW, usually non-domestic ...",32.0,400.0,22.0,20.0,{'Description': 'Alternating Current - Three P...,CAD14D5B-7DED-4534-A10A-650354BD9C09,Type 2 (Socket Only),2.0,NaN
4,91504,25,"{'FormalName': 'IEC 62196-2 Type 2', 'IsDiscon...",NaN,50,"{'IsOperational': True, 'IsUserSelectable': Tr...",2.0,"{'Comments': 'Over 2 kW, usually non-domestic ...",16.0,230.0,3.7,10.0,{'Description': 'Alternating Current - Single ...,B531CB42-F2C5-4C9A-99AD-94F6D02DEE89,Type 2 (Socket Only),1.0,NaN


In [26]:
len(arr2)

334

In [27]:
merged_df = arr1.merge(arr2, on='UUID', how='left')

In [28]:
len(merged_df)

334

In [29]:
merged_df.to_csv('02_charging_stations_final.csv', index=False)

In [30]:
merged_df['Name'].unique()

array(['Type 2 (Socket Only)', 'Tesla (Model S/X)', 'CCS (Type 2)',
       'CHAdeMO', 'Type 1 (J1772)', 'BS1363 3 Pin 13 Amp',
       'NACS / Tesla Supercharger'], dtype=object)

In [31]:
# Define a dictionary to map colors to categories
color_map = {
    'Type 1 (J1772)': 'orange',
    'Type 2 (Socket Only)': 'blue',
    'CCS (Type 2)': 'red',
    'Tesla (Model S/X)': 'green',
    'CHAdeMO': 'gray',
    'BS1363 3 Pin 13 Amp': 'purple',
    'Tesla Supercharger': 'darkgreen'
}

# Create a map centered around Dublin to display the EV charging points
map_osm = folium.Map(location=[latitude, longitude], zoom_start=12)

# Add markers for each charging station location with a different marker color for different charge type
for index, row in merged_df.iterrows():
    name = row['Name'] + ', ' + row['Title'] + ', ' + row['AddressLine1'] + ', ' + str(row['Town'])
    lon = row['Longitude']
    lat = row['Latitude']
    
    category = row['Name']
    marker_color = color_map.get(category, 'gray')  # Use 'gray' if category not found in the color map

    # marker = folium.Marker(location=[lat, lon], popup=name)
    # marker = folium.CircleMarker(location=[lat, lon], radius=5, color=marker_color, fill=True, fill_color=marker_color, fill_opacity=1, popup=name)
    icon = folium.Icon(color=marker_color, icon='map-marker')  # Use the 'map-marker' icon with the specified color

    marker = folium.Marker(location=[lat, lon], icon=icon, popup=name)

    marker.add_to(map_osm)

# Display the map
map_osm